<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/nlp/fine-tuned-llm/wiseai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Psy AI
Psy AI is a Llama3-8b model instruction fine-tuned on depression dataset and is meant to help improve mental well-being.

In future iterations it is meant to be trained on multiple philosophical and psychological datasets in order to provide multifaceted answers to complex mental health issues.



# Datasets

Philosophy datasets (* for future training)
* https://www.kaggle.com/datasets/christopherlemke/philosophical-texts
* https://www.workwithdata.com/object/philosophy-science-complete-a-text-on-traditional-problems-schools-thought-book-by-edwin-h-c-hung-0000
* https://www.kaggle.com/datasets/christopherlemke/philosophy-authors-writings-german
* https://www.workwithdata.com/object/philosophical-inquiries-an-introduction-to-problems-philosophy-book-by-nicholas-rescher-0000
* https://www.workwithdata.com/object/roman-stoicism-book-by-edward-vernon-arnold-1857
* https://www.workwithdata.com/object/wisdom-energy-basic-buddhist-teachings-book-by-thubten-yeshe-1935

Psychology and mental health datasets

* Kaggle Depression data for chatbot https://www.kaggle.com/datasets/nupurgopali/depression-data-for-chatbot
* Kaggle Psychometrics dataset https://www.kaggle.com/discussions/general/304994
* Psychometric tests dataset https://ieee-dataport.org/documents/psychometric-tests-dataset
* Psychometric NLP https://paperswithcode.com/dataset/psychometric-nlp
* Reddit mental health dataset https://zenodo.org/record/3941387
* Reddit mental disorders identification https://www.kaggle.com/datasets/kamaruladha/mental-disorders-identification-reddit-nlp
* Kaggle Mental Health Conversational Data https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data
* Kaggle Mental Health FAQ for Chatbot https://www.kaggle.com/narendrageek/mental-health-faq-for-chatbot/code
* A human consciousness questionnaire dataset https://data.mendeley.com/datasets/69p62ksdh6
* paperswithcode Self-reported Mental Health Diagnoses https://paperswithcode.com/dataset/smhd
* paperswithcode Mental Health Summarization Dataset https://paperswithcode.com/dataset/mentsum
* HuggingFace psychology dataset https://huggingface.co/datasets/samhog/psychology-10k


In [0]:
!pip install -U -q gradio
!pip install -U -q datasets
!pip install -U -q bitsandbytes
!pip install -U -q git+https://github.com/huggingface/transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -U -q trl
!pip install -U -q torch torchvision
!pip install -U -q accelerate
!pip install -U -q flash-attn

!pip install -U -q evaluate
!pip install -U -q rouge_score
!pip install -U -q optuna

!pip install -U -q urllib3==1.26.7
!pip install -U -q mlflow-skinny==2.12.1

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclassic 0.5.2 requires prometheus-client, which is not installed.
jupyter-server 1.23.4 requires prometheus-client, which is not installed.
databricks-feature-engineering 0.2.1 requires pyspark<4,>=3.1.2, which is not installed.
ydata-profiling 4.2.0 requires pydantic<2,>=1.8.1, but you have pydantic 2.7.1 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
spacy 3.7.2 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
google-auth 2.21.0 requires urllib3<2.0, but you have urllib3 2.2.1 which is incompatible.
databricks-sdk 0.1.6 requires requests<2.29.0,>=2.28.1, but you have requests 2.31.0 which is incompatible.
boto

In [0]:
dbutils.library.restartPython()

In [0]:
from datetime import datetime
from datasets import load_dataset
import json
import yaml
import gradio as gr
import torch
import transformers
from transformers import GenerationConfig, Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel, AutoPeftModelForCausalLM
import numpy as np
from evaluate import load
import optuna
import mlflow

import warnings
warnings.filterwarnings('ignore')

2024-05-09 12:54:30.191830: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 12:54:30.191911: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 12:54:30.237142: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 12:54:30.981220: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Dataset instruction transformation

Depression dataset with 51 q&a entries was taken for mvp to see if model is indeed training.

In [0]:
depression_dataset = load_dataset("vitaliy-sharandin/depression-instruct")

Generating train split:   0%|          | 0/51 [00:00<?, ? examples/s]

First, we modify our dataset to Alpaca format and create two datasets. One - for testing and evaluation and second one for inference, where responses are not available in formatted prompt.

In [0]:
def formatting_func_train(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n"
      f"{example['response']}")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response:\n"
      f"{example['response']}")

  return {"text" : input_prompt}

def formatting_func_test(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response: \n")

  return {"text" : input_prompt}

In [0]:
formatted_depression_dataset_train = depression_dataset.map(formatting_func_train)
formatted_depression_dataset_test = depression_dataset.map(formatting_func_test)

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

# Model load

We are loading our Llama2 model in 4bit quantized form as well as applying Lora for peft training.

In [0]:
model_name = "NousResearch/Meta-Llama-3-8B-Instruct"

def get_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"
  return tokenizer

def get_model():

  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  qlora_config = LoraConfig(lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM")

  base_training_model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config=bnb_config,
      device_map = {"": 0},
      offload_folder = f"output/{model_name}"
  )
  
  base_training_model.gradient_checkpointing_enable()
  base_training_model = prepare_model_for_kbit_training(base_training_model)
  base_training_model = get_peft_model(base_training_model, qlora_config)
  return base_training_model.to('cuda')

base_training_model = get_model()
tokenizer = get_tokenizer()

torch.manual_seed(42)
print(base_training_model)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=F

# Model instruction fine-tuning

Here we are defining inference function which returns bleu, rouge and f1 metrics after comparison of predicted and reference responses. My tests have shown that standard trainer `compute_metrics` method is quite inefficient and is not quite suitable for instruction fine-tuning during manual observations or generated results.

In [0]:
model_trained_checkpoint = f"{model_name}-peft-tuned"
catalog = "model_registry"
schema = "default"
model_name = "psy-ai"

def run_inference(model, test_dataset, test_size=None):

  if test_size:
    test_dataset = test_dataset['train'].select(range(test_size))
  else:
    test_dataset = test_dataset['train']

  inputs = test_dataset['text']
  reference_responses = test_dataset['response']

  decoded_predictions = []
  for prompt in inputs:
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    prediction = pipe(prompt, max_length=200, top_p=0.9, temperature=0.9, num_return_sequences=1, return_full_text=False)[0]['generated_text']
    decoded_predictions.append(prediction[:str(prediction).find("###")])

  for input, pred, label in zip(inputs[:3], decoded_predictions[:3], reference_responses[:3]):
    print("[Input]:\n\n", input)
    print("[Prediction]:\n\n", pred)
    print("[Reference response]:\n\n", label)
    print("----\n\n")

  bleu = load("bleu")
  bleu_results = bleu.compute(predictions=decoded_predictions, references=reference_responses)

  rouge = load('rouge')
  rouge_results = rouge.compute(predictions=decoded_predictions, references=reference_responses)

  f1 = 2 * (bleu_results['bleu'] * rouge_results['rouge1']) / (bleu_results['bleu'] + rouge_results['rouge1'])

  scores = {
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "f1": f1
    }

  return scores

def bleu_rouge_f1_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=-1)

  labels = [[idx for idx in label if idx != -100] for label in labels]
  predictions = [[idx for idx in prediction if idx != -100] for prediction in predictions]

  decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  bleu = load("bleu")
  bleu_results = bleu.compute(predictions=decoded_predictions, references=decoded_labels)

  rouge = load('rouge')
  rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

  f1 = 2 * (bleu_results['bleu'] * rouge_results['rouge1']) / (bleu_results['bleu'] + rouge_results['rouge1'])

  scores = {
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "f1": f1
    }

  return scores

def fine_tune(model, tokenizer, train_dataset, eval_dataset, metrics_func, only_evaluate=False):
  
  supervised_finetuning_trainer = SFTTrainer(model=model,
                                            train_dataset=train_dataset,
                                            eval_dataset=eval_dataset,
                                            args=TrainingArguments(
                                                report_to="mlflow",
                                                run_name=f"{model_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}",
                                                output_dir="output/training_results",
                                                per_device_train_batch_size=8,
                                                per_device_eval_batch_size=1,
                                                gradient_accumulation_steps=2,
                                                gradient_checkpointing=True,
                                                ddp_find_unused_parameters=False,
                                                optim="paged_adamw_8bit",
                                                save_steps=1000,
                                                logging_steps=30,
                                                learning_rate=2e-4,
                                                weight_decay=0.001,
                                                fp16=False,
                                                bf16=False,
                                                max_grad_norm=0.3,
                                                max_steps=-1,
                                                warmup_ratio=0.3,
                                                group_by_length=True,
                                                lr_scheduler_type="constant"
                                            ),
                                            dataset_text_field="text",
                                            max_seq_length=2048,
                                            compute_metrics=metrics_func,
                                            data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))

  if only_evaluate:
    mlflow.autolog(disable=True)
    return supervised_finetuning_trainer.evaluate()

  else:
    mlflow.autolog(disable=False)
    
    mlflow.set_experiment('/Shared/psy-ai')
    with mlflow.start_run() as run:

      supervised_finetuning_trainer.train()

      model_info = mlflow.transformers.log_model(
                    transformers_model={"model": supervised_finetuning_trainer.model, "tokenizer": tokenizer},
                    task="text-generation",
                    save_pretrained=False,
                    artifact_path=model_trained_checkpoint)
      
      mlflow.set_registry_uri("databricks-uc")
      mlflow.register_model(
          model_uri=model_info.model_uri,
          name=f"{catalog}.{schema}.{model_name}"
      )

## Evaluating base model

In [0]:
# params = {
#     "model": base_training_model,
#     "tokenizer": tokenizer,
#     "train_dataset": formatted_depression_dataset_train["train"],
#     "eval_dataset": formatted_depression_dataset_train["train"],
#     "metrics_func": bleu_rouge_f1_metrics,
#     "only_evaluate": True
# }

# fine_tune(**params)

Evaluation metrics are quite low before training.

Let's train the model and see how metrics and responses change.

## Fine-tuning base model and registering it to Unity Catalogue model registry


In [0]:
params = {
    "model": base_training_model,
    "tokenizer": tokenizer,
    "train_dataset": formatted_depression_dataset_train["train"],
    "eval_dataset": formatted_depression_dataset_train["train"],
    "metrics_func": bleu_rouge_f1_metrics
}

fine_tune(**params)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024/05/09 13:20:50 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of transformers. If you encounter errors during autologging, try upgrading / downgrading transformers to a supported version, or try upgrading MLflow.
2024/05/09 13:20:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2024/05/09 13:20:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2024/05/09 13:20:50 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2024/05/09 13:20:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2024/05/09 13:20:50 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.


Step,Training Loss


2024/05/09 13:23:22 INFO mlflow.transformers: Overriding save_pretrained to False for PEFT models, following the Transformers behavior. The PEFT adaptor and config will be saved, but the base model weights will not and reference to the HuggingFace Hub repository will be logged instead.
2024/05/09 13:23:22 INFO mlflow.transformers: Skipping saving pretrained model weights to disk as the save_pretrained is set to False. The reference to HuggingFace Hub repository NousResearch/Meta-Llama-3-8B-Instruct will be logged instead.


Uploading artifacts:   0%|          | 0/14 [00:00<?, ?it/s]

2024/05/09 13:23:23 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
Registered model 'model_registry.default.psy-ai' already exists. Creating a new version of this model...


...

2024/05/09 13:23:27 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the disk and cpu.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File /local_disk0/.ephemeral_nfs/envs/pythonEnv-cda8f181-0cb6-43ba-a892-bf05d3d9d1d4/lib/python3.10/site-packages/mlflow/store/_unity_catalog/registry/rest_store.py:627, in UcModelRegistryStore._download_model_weights_if_not_saved(self, local_model_path)
    626 try:
--> 627     mlflow.transformers.persist_pretrained_model(local_model_path)
    628 except Exception as e:

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-cda8f181-0cb6-43ba-a892-bf05d3d9d1d4/lib/python3.10/site-packages/mlflow/transformers/__init__.py:1080, in persist_pretrained_model(model_uri)
   1079 local_model_path = artifact_repo.download_artifacts(artifact_path, dst_path=tmp_dir.path())
-> 1080 pipeline = load_model(local_model_path, return_type="pipeline")
   1082 # Update MLModel flavor config

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-cda8f181-0cb6-43ba-a89

# Deploying model to Databricks Serving Endpoints


# Gradio demo in Hugging Face Spaces

https://huggingface.co/spaces/vitaliy-sharandin/WiseAI

## Local launch
Requirements:
* Choose free T4 GPU in Colab environment settings
* Run `pip install` and `imports` cells at the beginning before launching Gradio.

In [0]:
model = AutoModelForCausalLM.from_pretrained(
      'vitaliy-sharandin/wiseai',
      load_in_8bit=True,
      device_map = {"": 0}
  )
tokenizer = AutoTokenizer.from_pretrained('vitaliy-sharandin/wiseai')

pipe = pipeline('text-generation', model=model,tokenizer=tokenizer)

def generate_text(instruction, input):
  if not instruction.strip():
    return str('The instruction field is required.')

  if instruction.strip() and input.strip():
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{instruction}\n\n"
      "### Input:\n"
      f"{input}\n\n"
      f"### Response: \n")
  else :
    input_prompt = (f"Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n"
        f"{instruction}\n\n"
        f"### Response: \n")
  result = pipe(input_prompt, max_length=200, top_p=0.9, temperature=0.9, num_return_sequences=1, return_full_text=False)[0]['generated_text']
  return result[:str(result).find("###")]

iface = gr.Interface(fn=generate_text, inputs=[gr.Textbox(label="Instruction"),
                                               gr.Textbox(label="Additional Input")],
                     outputs=gr.Textbox(label="Response"))
iface.launch()

---------------------------------------------------------------------------
RestException                             Traceback (most recent call last)
File <command-1780649499146088>, line 9
      1 params = {
      2     "model": base_training_model,
      3     "tokenizer": tokenizer,
   (...)
      6     "metrics_func": bleu_rouge_f1_metrics
      7 }
----> 9 fine_tune(**params)

File <command-1780649499146083>, line 124, in fine_tune(model, tokenizer, train_dataset, eval_dataset, metrics_func, only_evaluate)
    117 model_info = mlflow.transformers.log_model(
    118               transformers_model={"model": supervised_finetuning_trainer.model, "tokenizer": tokenizer},
    119               task="text-generation",
    120               save_pretrained=True,
    121               artifact_path=model_trained_checkpoint)
    123 mlflow.set_registry_uri("databricks-uc")
--> 124 mlflow.register_model(
    125     model_uri=model_info.model_uri,
    126     name=f"{catalog}.{schema}.{m